In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [3]:
analiza[analiza.OPIS.str.contains("GOYA")]

,KOD,OPIS,MAX,CZY_BRYLA,obj,BRYLA_GEN,PREFEROWANY_CZAS_KJ,PREFEROWANY_CZAS_PAKOWANIA,RODZINA_NAZWA,SALDO,...,WOLNE_NIE_SPAK,MAX_obj,ZAMOWIONE_obj,CZEKA_NA_SPAKOWANIE_obj,CZESCIOWO_DOSTARCZONE_obj,SALDO_obj,WOLNE_obj,WOLNE_NIE_SPAK_obj,DO_ZAM_SZT,DO_ZAM_obj


In [ ]:
Podsumowanie_paczek_i_Pw("")

In [ ]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"])
# br.to_excel("BRAKI.xlsx")

In [ ]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

In [ ]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

In [ ]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)



In [ ]:
arp

In [ ]:
arp>20

In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



In [ ]:
# Zapis_danych_do_Archiwum(24x, analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")